In [12]:
import pandas as pd
import mysql.connector

### Data Insertion in Project Table

In [15]:
df = pd.read_csv('Crowdfunding_projects.csv')
df.head()

,id,state,country,creator_id,location_id,category_id,goal,pledged,currency,usd_pledged,backers_count,cal_id
0,24080365,failed,US,1439247114,2488042.0,293,5000.0,0.0,USD,0.0,0,1
1,27131878,failed,US,680525996,2466256.0,10,5000.0,0.0,USD,0.0,0,2
2,30525634,failed,US,222229854,2458410.0,276,5000.0,0.0,USD,0.0,0,3
3,31977817,failed,US,1978657574,2523194.0,48,5000.0,0.0,USD,0.0,0,4
4,33517733,failed,US,1016683395,2433227.0,18,5000.0,0.0,USD,0.0,0,5


In [17]:
if df is not None:
    print(df.head())  

   
    df['location_id'] = df['location_id'].fillna(0)  
    
    print(f"NaN values in 'location_id' column: {df['location_id'].isna().sum()}")
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    conn = mysql.connector.connect(
        host="localhost",        
        user="root",    
        password="Praja@2021",
        database="crowd_funding" 
    )

    cursor = conn.cursor()
    
    insert_query = '''
    INSERT INTO projects (ProjectID, state, country, creator_id, location_id, category_id, goal, pledged, currency, usd_pledged, backers_count, calendar_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    
    batch_size = 10000
    for i in range(0, len(data_tuples), batch_size):
        cursor.executemany(insert_query, data_tuples[i:i+batch_size])
        conn.commit()  
        
    cursor.close()
    conn.close()

    print("Data inserted successfully!")
else:
    print("Error: DataFrame is None. Check the file path or the content of the CSV.")


         id   state country  creator_id  location_id  category_id    goal  \
0  24080365  failed      US  1439247114    2488042.0          293  5000.0   
1  27131878  failed      US   680525996    2466256.0           10  5000.0   
2  30525634  failed      US   222229854    2458410.0          276  5000.0   
3  31977817  failed      US  1978657574    2523194.0           48  5000.0   
4  33517733  failed      US  1016683395    2433227.0           18  5000.0   

   pledged currency  usd_pledged  backers_count  cal_id  
0      0.0      USD          0.0              0       1  
1      0.0      USD          0.0              0       2  
2      0.0      USD          0.0              0       3  
3      0.0      USD          0.0              0       4  
4      0.0      USD          0.0              0       5  
NaN values in 'location_id' column: 0
Data inserted successfully!


### Data Insertion of Calendar_Table

In [55]:
df_cal = pd.read_csv("Calendar_table.csv")

In [57]:
df_cal.shape

(365892, 8)

In [61]:
print(df_cal.head())

   Year  Month  Day  Start_Date Quarter  Financial Quarter  id    End_Date
0  2017      2    1  02-01-2017      Q1                  5   1  03-03-2017
1  2015      4    1  04-01-2015      Q2                  1   2  05-01-2015
2  2016      3    8  03-08-2016      Q1                  5   3  04-08-2016
3  2012      7   25  07/25/2012      Q3                  2   4  08/30/2012
4  2015      3   31  03/31/2015      Q1                  5   5  04-10-2015


In [65]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

data = pd.read_csv("Calendar_table.csv")

# 2. Convert 'Start_Date' and 'End_Date' to proper datetime format
data['Start_Date'] = pd.to_datetime(data['Start_Date'], errors='coerce')
data['End_Date'] = pd.to_datetime(data['End_Date'], errors='coerce')


data['Start_Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])


data['Month'] = data['Month'].apply(lambda x: x if 1 <= x <= 12 else None)
data['Day'] = data['Day'].apply(lambda x: x if 1 <= x <= 31 else None)

# 5. Define a function to calculate the quarter based on the month
def get_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    elif month in [10, 11, 12]:
        return 'Q4'
    return None

data['Quarter'] = data['Month'].apply(get_quarter)

# 6. Define a function to calculate the financial quarter based on month
def get_financial_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    elif month in [10, 11, 12]:
        return 4
    return None

data['Financial Quarter'] = data['Month'].apply(get_financial_quarter)


print("DataFrame to be inserted:")
print(data)

# 8. Prepare data for insertion into MySQL (include 'id' column)
data_tuples = [tuple(x) for x in data[['id', 'Year', 'Month', 'Start_Date', 'Quarter', 'Financial Quarter', 'End_Date']].to_numpy()]

# 9. Establish MySQL connection
try:
    conn = mysql.connector.connect(
        host="localhost",        
        user="root",             
        password="Praja@2021",   
        database="crowd_funding" 
    )

    if conn.is_connected():
        print("Connected to MySQL")

        # Create a cursor object
        cursor = conn.cursor()

        # Define the insert query (match with your table schema)
        insert_query = '''
        INSERT INTO calendar_table (id, year, month, start_date, quarter, financial_quarter, end_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        '''
        
        batch_size = 1000 
        for i in range(0, len(data_tuples), batch_size):
            try:
                cursor.executemany(insert_query, data_tuples[i:i + batch_size])
                conn.commit()
                print(f"Batch {i} inserted successfully.")
            except mysql.connector.Error as err:
                print(f"Error during batch insertion: {err}")

    else:
        print("Failed to connect to MySQL.")

except Error as e:
    print(f"Error: {e}")
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed.")


DataFrame to be inserted:
        Year  Month  Day Start_Date Quarter  Financial Quarter      id  \
0       2017      2    1 2017-02-01      Q1                  1       1   
1       2015      4    1 2015-04-01      Q2                  2       2   
2       2016      3    8 2016-03-08      Q1                  1       3   
3       2012      7   25 2012-07-25      Q3                  3       4   
4       2015      3   31 2015-03-31      Q1                  1       5   
...      ...    ...  ...        ...     ...                ...     ...   
365887  2017      8   13 2017-08-13      Q3                  3  365888   
365888  2016      1   21 2016-01-21      Q1                  1  365889   
365889  2018      3   21 2018-03-21      Q1                  1  365890   
365890  2016     10   25 2016-10-25      Q4                  4  365891   
365891  2015     10   29 2015-10-29      Q4                  4  365892   

         End_Date  
0      2017-03-03  
1      2015-05-01  
2      2016-04-08  
3    

In [73]:
data.shape

(365892, 8)